In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sympy as sp
from scipy.integrate import odeint
import os

In [246]:
data = 'altimeter_data.csv'
df = pd.read_csv(data,header=None, names=['t','yk','hk','sk','cbk'])
df = df.dropna()

In [247]:
# Create symbolic representations of variables to compute jacobians using SymPy
h, s, d, delt, cb, rn, hr, hn, re, gn = sp.symbols(r'h s d \Delta{t} C_b \rho_0 h_\rho h_0 R_E g_0') 

In [248]:
P0 = np.diag([100**2, 10**2, 1])
Q = np.diag([10**2, 10**2, .05**2])
R = 100**2

### Dynamics Model

In [249]:
def propogate(x):
    '''Function to compute dynamics of state variables'''
    dt = 0.5
    rho_0 = .0765
    g0 = 32.2
    h_rho = 30000
    Re = 20902260
    h = x[0]
    s = x[1]
    cb = x[2]

    hkp1 = h + dt*s
    skp1 = s + dt*(((rho_0*s**2)/(2*cb)*np.exp(-h/h_rho)) - (g0*((Re/(Re+h))**2)))
    cbkp1 = cb
    return np.array([hkp1, skp1, cbkp1])

In [250]:
def jacobian_F(x):
    '''Function that computes analytical jacobian of F at state x'''
    F_discrete_symbolic = sp.Matrix([[h + delt*s],
                                [s + (delt*(((rn*s**2)/(2*cb)*sp.exp(-h/hr)) - (gn*((re/(re+h))**2))))],
                                [cb]])
    constants_mapper = dict(zip([delt,rn,gn,hr,re],[.5, .0765, 32.2, 30000, 20902260])) # Use given constant values to map to symbolic 
    F_jacobian_symbolic = F_discrete_symbolic.jacobian([h, s, cb]).subs(constants_mapper) # take jacobian and substitute in numerical values
    
    x = (np.array(x)).flatten()
    h_in = x[0]
    s_in = x[1]
    cb_in = x[2]
    F_jacobian = F_jacobian_symbolic.subs({h:h_in, s:s_in, cb:cb_in})
    return np.array(F_jacobian).astype(np.float64)

### Measurement Model

In [251]:
def get_measurement(x):
    h_in = x[0]
    s_in = x[1]
    cb_in = x[2]
    d = 100000
    out = np.sqrt(d**2 + h_in**2)
    return out

In [384]:
def jacobian_H(x):
    h_in = x[0]
    s_in = x[1]
    cb_in = x[2]
    measurement_function = sp.Matrix([sp.sqrt(d**2 + h**2)])
    jacobian_H = measurement_function.jacobian([h, s, cb]).subs({h:h_in, s:s_in, d:100000})
    return np.array(jacobian_H).astype(np.float64).flatten()

### Propogate Dynamics

In [385]:
x0 = np.array([400000, -2000, 20])

In [386]:
n = 600
res = np.zeros((n, 3))
mmts = np.zeros((n,1))
for k in range(n):
    if k == 0:
        x = x0
        P = P0
        meas = get_measurement(x)
        res[k] = x
        mmts[k] = meas
        continue
    hk, sk, cbk = propogate(x)
    x = np.array([hk, sk, cbk])
    res[k] = x
    mmts[k] = get_measurement(x)

In [387]:
def predict(x_in, P_in, Q):
    F_jacobian = jacobian_F(x_in)
    x_k_km1 = propogate(x_in)
    P_k_km1 = F_jacobian @ P_in @ F_jacobian.T + Q
    return x_k_km1, P_k_km1

def update(xhat, Phat, z, R):
    H_jacobian = jacobian_H(xhat)
    hx = get_measurement(xhat)
    print(hx)
    y_innov = z - hx
    S = 1/(H_jacobian @ Phat @ H_jacobian.T + R)
    K = np.dot(Phat, H_jacobian.T).dot(S)
    x_updated = xhat + np.dot(K, y_innov)
    imkh = np.eye(3) - np.dot(K, H_jacobian)
    krk = np.dot(K, R).dot(K.T)
    P_updated = imkh.dot(Phat).dot(imkh.T) + krk
    return x_updated, P_updated

In [388]:
n = 600
res = np.zeros((n, 3))
for k in range(n):
    if k == 0:
        x = x0
        P = P0
        res[k] = x
        continue
    xkp1, Pkp1 = predict(x, P, Q)
    xu, Pu = update(xkp1, Pkp1, df.loc[k,'yk'], R)
    x = xu
    P = Pu
    res[k] = x

411340.4915638625
408524.998471132
406591.4401729131
404992.0386427645
403659.24385972944
402319.1154579334
401176.24407155946
400085.7391335546
398993.93444801174
397973.4446140266
396928.3828640597
395908.8405282809
394948.7684667566
393974.0763993745
392991.6349229221
392037.712715047
391005.57329301094
390052.30360650446
389081.3545554075
388068.20858631463
387047.7438284576
386051.08298678754
384975.21340041386
383840.8183605831
382751.91201445594
381708.5372189223
380563.19164311874
379383.6360019068
378295.68411974894
377113.57586048916
375894.6919166602
374725.5507788282
373535.19646596006
372401.8006220039
371259.40295340796
370139.4466198884
369008.0643624889
367849.8391012368
366605.4511883924
365327.68992495176
364160.37680698646
362918.39861743775
361623.3169261282
360475.19262620364
359045.59954031446
357854.1646167136
356639.0321386107
355371.2499950295
354521.8885544473
353049.50476196205
351685.04722153995
350330.53114235104
349265.4189782714
347942.84323053516
346631.

110445.02505604172
110010.36667811126
110632.65122041825
110313.56319056613
110028.43424989002
110373.80154865321
110081.38858507248
109769.59206473875
110084.42661359232
110084.3204330929
109986.03257471246
109657.66675781814
109653.79501924812
109702.92976441055
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


C:\Users\Benjamin Lucke\AppData\Local\Temp\ipykernel_17648\442119922.py:4: RuntimeWarning: invalid value encountered in matmul
  P_k_km1 = F_jacobian @ P_in @ F_jacobian.T + Q
C:\Users\Benjamin Lucke\AppData\Local\Temp\ipykernel_17648\2525449403.py:14: RuntimeWarning: invalid value encountered in cast
  return np.array(F_jacobian).astype(np.float64)
C:\Users\Benjamin Lucke\AppData\Local\Temp\ipykernel_17648\2852444922.py:7: RuntimeWarning: invalid value encountered in cast
  return np.array(jacobian_H).astype(np.float64).flatten()


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
